# Ejemplo de patrones: TopN

En este ejercicio partimos de un dataset que contiene posts enviados a un foro de Internet. Se pide filtrar los posts, devuelva los N posts cuyos contenidos sean los más largos.

Un post contiene una serie de campos separados por tabuladores. Los campos son los siguientes: 

"id", "title", "tagnames", "author_id", "body", "node_type", "parent_id", "abs_parent_id", "added_at", "score", "state_string", "last_edited_id", "last_activity_by_id", "last_activity_at", "active_revision_id", "extra", "extra_ref_id", "extra_count", "marked" 

Para este ejercicio solamente nos interesa el campo "body" que contiene el texto del post. 

In [1]:
! mkdir -p ejemplo-patrones/topN

In [2]:
import os
os.chdir("/media/notebooks/ejemplo-patrones/topN")

In [3]:
! pwd

/media/notebooks/ejemplo-patrones/topN


In [4]:
%%writefile mapper.py
#!/usr/bin/env python

import sys
import csv

reader = csv.reader(sys.stdin, delimiter='\t')

mykey = None # inventado

topN = 2

#Una lista que incluye cada linea y su longitud en forma de sublista
lineWithLength=[]

# PON TU CODIGO AQUÍ

Overwriting mapper.py


In [5]:
%%writefile reducer.py
#!/usr/bin/env python

import sys
import csv

topN_Value = 2

lineWithLength=[]

# PON TU CODIGO AQUÍ

Overwriting reducer.py


In [6]:
! hdfs dfs -ls /tmp/forumdata

Found 3 items
-rw-r--r--   3 root supergroup       1774 2019-08-12 08:03 /tmp/forumdata/forum1.tsv
-rw-r--r--   3 root supergroup  120313812 2019-08-12 08:03 /tmp/forumdata/forum_node.tsv
-rw-r--r--   3 root supergroup     530989 2019-08-12 08:03 /tmp/forumdata/forum_users.tsv


La siguiente celda realiza una prueba con un fichero de posts corto llamado forum1.tsv. Asi se ve facilmente que el codigo funciona bien.

In [11]:
! hdfs dfs -rm /tmp/salida-topNTest/*
! hdfs dfs -rmdir /tmp/salida-topNTest

rm: `/tmp/salida-topNTest/*': No such file or directory
rmdir: `/tmp/salida-topNTest': No such file or directory


In [12]:
! hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files mapper.py,reducer.py  -mapper mapper.py -reducer reducer.py \
-input /tmp/forumdata/forum1.tsv -output /tmp/salida-topNTest

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob814665353921995048.jar tmpDir=null
19/08/12 09:58:09 INFO client.RMProxy: Connecting to ResourceManager at yarnmaster/172.21.0.3:8032
19/08/12 09:58:09 INFO client.RMProxy: Connecting to ResourceManager at yarnmaster/172.21.0.3:8032
19/08/12 09:58:10 INFO mapred.FileInputFormat: Total input paths to process : 1
19/08/12 09:58:10 INFO mapreduce.JobSubmitter: number of splits:2
19/08/12 09:58:10 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1565590814059_0017
19/08/12 09:58:10 INFO impl.YarnClientImpl: Submitted application application_1565590814059_0017
19/08/12 09:58:10 INFO mapreduce.Job: The url to track the job: http://yarnmaster:8088/proxy/application_1565590814059_0017/
19/08/12 09:58:10 INFO mapreduce.Job: Running job: job_1565590814059_0017
19/08/12 09:58:15 INFO mapreduce.Job: Job job_1565590814059_0017 running in uber mode : false
19/08/12 09:58:15 INFO mapreduce.Job:

In [13]:
! hdfs dfs -tail /tmp/salida-topNTest/part-00000

['0206', 'Titulo', 'tags', '9191', 'Este es un mensaje de 5\\n  lineas\\n pero\\n solo \\n 1 frase.', 'question', '\\N', '\\N', '2012-02-27 15:09:11.184434+00', '0', '', '\\N', '100003268', '2012-02-27 15:09:11.184434+00', '9322', '\\N', '\\N', '106', 'f']	
['0207', 'Titulo', 'tags', '9191', 'Este es\\n un mensaje de 6\\n lineas\\n pero \\nsolo 1 \\nfrase.', 'question', '\\N', '\\N', '2012-02-27 15:09:11.184434+00', '0', '', '\\N', '100003268', '2012-02-27 15:09:11.184434+00', '9322', '\\N', '\\N', '106', 'f']	


In [14]:
! hdfs dfs -rm /tmp/salida-topN/*
! hdfs dfs -rmdir /tmp/salida-topN

rm: `/tmp/salida-topN/*': No such file or directory
rmdir: `/tmp/salida-topN': No such file or directory


In [15]:
! hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /tmp/forumdata/forum_node.tsv -output /tmp/salida-topN

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob1439647105550458621.jar tmpDir=null
19/08/12 09:59:15 INFO client.RMProxy: Connecting to ResourceManager at yarnmaster/172.21.0.3:8032
19/08/12 09:59:15 INFO client.RMProxy: Connecting to ResourceManager at yarnmaster/172.21.0.3:8032
19/08/12 09:59:15 INFO mapred.FileInputFormat: Total input paths to process : 1
19/08/12 09:59:15 INFO mapreduce.JobSubmitter: number of splits:2
19/08/12 09:59:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1565590814059_0018
19/08/12 09:59:16 INFO impl.YarnClientImpl: Submitted application application_1565590814059_0018
19/08/12 09:59:16 INFO mapreduce.Job: The url to track the job: http://yarnmaster:8088/proxy/application_1565590814059_0018/
19/08/12 09:59:16 INFO mapreduce.Job: Running job: job_1565590814059_0018
19/08/12 09:59:21 INFO mapreduce.Job: Job job_1565590814059_0018 running in uber mode : false
19/08/12 09:59:21 INFO mapreduce.Job

In [17]:
! hdfs dfs -tail /tmp/salida-topN/part-00000

bundle/Contents/Resources/google_appengine/google/appengine/api/validation.py:73: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6<br>\n  self.message = message<br>\n/Users/George/Desktop/GoogleAppEngineLauncher.app/Contents/Resources/GoogleAppEngine-default.bundle/Contents/Resources/google_appengine/google/appengine/api/validation.py:77: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6<br>\n  return str(self.message)<br>\nError parsing yaml file:<br>\nInvalid object:<br>\nthreadsafe cannot be enabled with CGI handler: helloworld.py<br>\n  in "/Users/George/udacity1/app.yaml", line 9, column 24<br>\nIf deploy fails you might need to \'rollback\' manually.<br>\nThe "Make Symlinks..." menu option can help with command-line work.</em></strong> appcfg.py has finished with exit code 1 ***</p>', 'answer', '6010539', '6010539', '2012-04-28 22:00:17.081585+00', '0', '', '\\N', '100079746', '2012-04-28 22:00:17.081585+00', '6013214', '